In [1]:
## Importing libraries

import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


### Categorical - FE1

In [ ]:
## Importing data

data = pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/train.csv')
etd = pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/test.csv')

In [ ]:
not_used = []
cat_feat = []
target = 'target'

In [ ]:
features = [i for i in data.columns if i != target]

In [ ]:
from tqdm import tqdm

orig = [f'var_{i}' for i in range(200)]
has_one = [f'var_{i}_has_one' for i in range(200)]
has_zero = [f'var_{i}_has_zero' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]

for f in tqdm(orig):
    unique_v = etd[f].value_counts()
    unique_v = unique_v.index[unique_v == 1]
    etd[f + '_u'] = etd[f].isin(unique_v)

100%|██████████| 200/200 [00:03<00:00, 52.03it/s]


In [ ]:
etd['has_unique'] = etd[[f + '_u' for f in orig]].any(axis=1)
print(etd['has_unique'].sum())

100000


In [ ]:
real_samples = etd.loc[etd['has_unique'], orig]
ref = pd.concat([data, real_samples], axis=0)
print(ref.shape)
etd = etd.iloc[:, :201]

(300000, 202)


In [ ]:
for f in tqdm(orig):
    data[f + '_has_one'] = 0
    data[f + '_has_zero'] = 0
    f_1 = data.loc[data[target] == 1, f].value_counts()
    
    f_1_1 = set(f_1.index[f_1 > 1])
    f_0_1 = set(f_1.index[f_1 > 0])

    f_0 = data.loc[data[target] == 0, f].value_counts()
    f_0_0 = set(f_0.index[f_0 > 1])
    f_1_0 = set(f_0.index[f_0 > 0])
    
    data.loc[data[target] == 1, f + '_has_one'] = data.loc[data[target] == 1, f].isin(f_1_1).astype(int)
    data.loc[data[target] == 0, f + '_has_one'] = data.loc[data[target] == 0, f].isin(f_0_1).astype(int)

    data.loc[data[target] == 1, f + '_has_zero'] = data.loc[data[target] == 1, f].isin(f_1_0).astype(int)
    data.loc[data[target] == 0, f + '_has_zero'] = data.loc[data[target] == 0, f].isin(f_0_0).astype(int)

data.loc[:, has_one] = 2*data.loc[:, has_one].values + data.loc[:, has_zero].values

100%|██████████| 200/200 [00:25<00:00,  7.99it/s]


In [ ]:
for f in tqdm(orig):
    etd[f + '_has_one'] = 0
    etd[f + '_has_zero'] = 0
    f_1 = data.loc[data[target] == 1, f].unique()
    f_0 = data.loc[data[target] == 0, f].unique()
    etd.loc[:, f + '_has_one'] = etd[f].isin(f_1).astype(int)
    etd.loc[:, f + '_has_zero'] = etd[f].isin(f_0).astype(int)
    
etd.loc[:, has_one] = 2*etd.loc[:, has_one].values + etd.loc[:, has_zero].values

100%|██████████| 200/200 [00:06<00:00, 31.14it/s]


In [ ]:
for f in tqdm(orig):
    v = ref[f].value_counts()
    
    non_unique_v = v.index[v != 1]
    
    m_trd = data[f].isin(non_unique_v)
    data[f + '_not_unique'] = m_trd  * data[f] + (~m_trd) * data[f].mean()
    
    m_etd = etd[f].isin(non_unique_v)
    etd[f + '_not_unique'] = m_etd  * etd[f] + (~m_etd) * data[f].mean()
    
    data.loc[~m_trd, f + '_has_one'] = 4
    etd.loc[~m_etd, f + '_has_one'] = 4

100%|██████████| 200/200 [00:09<00:00, 21.28it/s]


In [ ]:
data['var_0_has_one'].value_counts()

1    126381
3     27409
4     23064
0     17291
2      5855
Name: var_0_has_one, dtype: int64

In [ ]:
print(data.shape)
print(etd.shape)

(200000, 802)
(200000, 801)


In [ ]:
data.to_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-categorical/train_fe1_categorical.csv", index=False)
etd.to_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-categorical/test_fe1_categorical.csv", index=False)

### Categorical - FE2

In [19]:
## Importing data

data = pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/train.csv')
etd = pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/test.csv')

In [20]:
## Reversing & dropping some features

drop_vars = ["var_7", "var_10", "var_17", "var_27", "var_29", "var_30", "var_38", "var_41", "var_46", "var_96", "var_100", "var_103", "var_126", "var_158", "var_185"]
reverse_list = ["var_0", "var_1", "var_2", "var_3", "var_4", "var_5", "var_6", "var_7", "var_8", "var_11", "var_15", "var_16", "var_18", "var_19", "var_22", "var_24", "var_25", "var_26", "var_27", "var_29", "var_32", "var_35", "var_37", "var_40", "var_41", "var_47", "var_48", "var_49", "var_51", "var_52", "var_53", "var_55", "var_60", "var_61", "var_62", "var_65", "var_66", "var_67", "var_69", "var_70", "var_71", "var_74", "var_78", "var_79", "var_82", "var_84", "var_89", "var_90", "var_91", "var_94", "var_95", "var_96", "var_97", "var_99", "var_103", "var_105", "var_106", "var_110", "var_111", "var_112", "var_118", "var_119", "var_125", "var_128", "var_130", "var_133", "var_134", "var_135", "var_137", "var_138", "var_140", "var_144", "var_145", "var_147", "var_151", "var_155", "var_157", "var_159", "var_161", "var_162", "var_163", "var_164", "var_167", "var_168", "var_170", "var_171", "var_173", "var_175", "var_176", "var_179", "var_180", "var_181", "var_184", "var_185", "var_187", "var_189", "var_190", "var_191", "var_195", "var_196", "var_199"]

for j in reverse_list:
    data[j] *= -1

for j in reverse_list:
    etd[j] *= -1

#data.drop(drop_vars, axis=1, inplace=True)
#etd.drop(drop_vars, axis=1, inplace=True)

In [21]:
not_used = []
cat_feat = []
target = 'target'
features = [i for i in data.columns if i != target]

In [22]:
from tqdm import tqdm

orig = [f'var_{i}' for i in range(200)]
has_one = [f'var_{i}_has_one' for i in range(200)]
has_zero = [f'var_{i}_has_zero' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]

for f in tqdm(orig):
    unique_v = etd[f].value_counts()
    unique_v = unique_v.index[unique_v == 1]
    etd[f + '_u'] = etd[f].isin(unique_v)

100%|██████████| 200/200 [00:03<00:00, 56.10it/s]


In [23]:
etd['has_unique'] = etd[[f + '_u' for f in orig]].any(axis=1)
print(etd['has_unique'].sum())

100000


In [24]:
real_samples = etd.loc[etd['has_unique'], orig]
ref = pd.concat([data, real_samples], axis=0)
print(ref.shape)
etd = etd.iloc[:, :186]

(300000, 202)


In [25]:
for f in tqdm(orig):
    data[f + '_has_one'] = 0
    data[f + '_has_zero'] = 0
    f_1 = data.loc[data[target] == 1, f].value_counts()
    
    f_1_1 = set(f_1.index[f_1 > 1])
    f_0_1 = set(f_1.index[f_1 > 0])

    f_0 = data.loc[data[target] == 0, f].value_counts()
    f_0_0 = set(f_0.index[f_0 > 1])
    f_1_0 = set(f_0.index[f_0 > 0])
    
    data.loc[data[target] == 1, f + '_has_one'] = data.loc[data[target] == 1, f].isin(f_1_1).astype(int)
    data.loc[data[target] == 0, f + '_has_one'] = data.loc[data[target] == 0, f].isin(f_0_1).astype(int)

    data.loc[data[target] == 1, f + '_has_zero'] = data.loc[data[target] == 1, f].isin(f_1_0).astype(int)
    data.loc[data[target] == 0, f + '_has_zero'] = data.loc[data[target] == 0, f].isin(f_0_0).astype(int)

data.loc[:, has_one] = 2*data.loc[:, has_one].values + data.loc[:, has_zero].values

100%|██████████| 200/200 [00:24<00:00,  8.04it/s]


In [26]:
for f in tqdm(orig):
    etd[f + '_has_one'] = 0
    etd[f + '_has_zero'] = 0
    f_1 = data.loc[data[target] == 1, f].unique()
    f_0 = data.loc[data[target] == 0, f].unique()
    etd.loc[:, f + '_has_one'] = etd[f].isin(f_1).astype(int)
    etd.loc[:, f + '_has_zero'] = etd[f].isin(f_0).astype(int)
    
etd.loc[:, has_one] = 2*etd.loc[:, has_one].values + etd.loc[:, has_zero].values

 92%|█████████▎| 185/200 [00:05<00:00, 30.97it/s]


KeyError: ignored

In [27]:
for f in tqdm(orig):
    v = ref[f].value_counts()
    
    non_unique_v = v.index[v != 1]
    
    m_trd = data[f].isin(non_unique_v)
    data[f + '_not_unique'] = m_trd  * data[f] + (~m_trd) * data[f].mean()
    
    m_etd = etd[f].isin(non_unique_v)
    etd[f + '_not_unique'] = m_etd  * etd[f] + (~m_etd) * data[f].mean()
    
    data.loc[~m_trd, f + '_has_one'] = 4
    etd.loc[~m_etd, f + '_has_one'] = 4

 92%|█████████▎| 185/200 [00:09<00:00, 20.00it/s]


KeyError: ignored

In [ ]:
data['var_0_has_one'].value_counts()

In [ ]:
print(data.shape)
print(etd.shape)

In [ ]:
data.to_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-categorical/train_fe2_categorical.csv", index=False)
etd.to_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-categorical/test_fe2_categorical.csv", index=False)